In [1]:
import pandas as pd
import json
import datetime

In [2]:
def format_date(date):

    formatted_date = '.'.join(str(date).split('-'))

    return formatted_date

In [3]:
raw_data_path = r"data\flights_data.json"
output_csv_path = r"data\flights.csv"

In [4]:
with open(raw_data_path) as f:
    data = json.load(f)["items"]

df = pd.DataFrame(data)
df["stops_num"] = df.stops.str["stops"]
max_stops = df["stops_num"].max()

df2 = pd.DataFrame(df.stops.str["stops_data"].tolist())

for i in range(max_stops):
    df[f"stop_{i+1}_duration"] = df2[i].str["duration"]
    df[f"stop_{i+1}_place"] = df2[i].str["place"]

df = df.drop('stops', axis=1)

In [5]:
today = datetime.date.today()
today = format_date(today)
df[df["date_readed"] == today].drop_duplicates().sort_values("price").head(10)

,date_readed,departure_date,arrival_date,departure_time,arrival_time,return_date,price,airline,duration,stops_num,stop_1_duration,stop_1_place,stop_2_duration,stop_2_place
664,2022.10.28,"Friday, February 3 2023","Saturday, February 4 2023",8:55 PM,9:20 AM,2023.02.08,579,easyJet,12 hr 25 min,1,7 hr 45 min,London,None,None
643,2022.10.28,"Friday, February 3 2023","Saturday, February 4 2023",8:55 PM,9:20 AM,2023.02.06,584,easyJet,12 hr 25 min,1,7 hr 45 min,London,None,None
653,2022.10.28,"Friday, February 3 2023","Saturday, February 4 2023",8:55 PM,9:20 AM,2023.02.07,588,easyJet,12 hr 25 min,1,7 hr 45 min,London,None,None
665,2022.10.28,"Friday, February 3 2023","Friday, February 3 2023",10:35 AM,7:00 PM,2023.02.08,589,easyJet,8 hr 25 min,1,3 hr 45 min,Gatwick,None,None
618,2022.10.28,"Thursday, February 2 2023","Thursday, February 2 2023",1:25 PM,10:25 PM,2023.02.07,601,Lufthansa,9 hr,1,5 hr 40 min,Munich,None,None
632,2022.10.28,"Thursday, February 2 2023","Thursday, February 2 2023",1:25 PM,10:25 PM,2023.02.08,601,Lufthansa,9 hr,1,5 hr 40 min,Munich,None,None
627,2022.10.28,"Thursday, February 2 2023","Thursday, February 2 2023",7:30 AM,12:05 PM,2023.02.08,601,Lufthansa,4 hr 35 min,1,1 hr 15 min,Munich,None,None
649,2022.10.28,"Friday, February 3 2023","Friday, February 3 2023",1:25 PM,5:35 PM,2023.02.07,601,Lufthansa,4 hr 10 min,1,50 min,Munich,None,None
661,2022.10.28,"Friday, February 3 2023","Friday, February 3 2023",1:25 PM,5:35 PM,2023.02.08,601,Lufthansa,4 hr 10 min,1,50 min,Munich,None,None
615,2022.10.28,"Thursday, February 2 2023","Thursday, February 2 2023",7:30 AM,12:05 PM,2023.02.07,601,Lufthansa,4 hr 35 min,1,1 hr 15 min,Munich,None,None


In [6]:
import os
from azure.storage.blob import BlobServiceClient

connect_str = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [7]:
container_name = "mol"
container_client= blob_service_client.get_container_client(container_name)

In [8]:
df.to_csv(output_csv_path)

blob_client = blob_service_client.get_blob_client(container=container_name, blob=output_csv_path)
print("\nUploading to Azure Storage as blob:\n\t" + output_csv_path)

with open(output_csv_path, "rb") as f:
    blob_client.upload_blob(f, overwrite=True)


Uploading to Azure Storage as blob:
	data\flights.csv
